# RegLin Synthesis

Using linear regression for the synthesis of leads on the ECG


## Essential Libraries

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression

## Loading the ECG

In [ ]:
samplePath = "samples/1005639.txt"

ecgHeaders = [
	"sample",
	"LI", 
	"LII", 
	"LIII", 
	"aVR", 
	"aVF", 
	"aVL",
	"V1",
	"V2",
	"V3",
	"V4",
	"V5",
	"V6"
]

ecgDf = pd.read_csv(samplePath)
ecgDf.columns = ecgHeaders
ecgDf = ecgDf.set_index("sample")

ecgDf.head()

## Comparative Ecg Plot Function

In [ ]:
def comparativeEcgPlot(ecg1, label1, ecg2, label2, title):
	plt.figure(figsize = (12, 6))

	plt.plot(ecg1, label = label1)
	plt.plot(ecg2, label = label2)

	plt.title(title)

	plt.xlabel("Amostra")
	plt.ylabel("Dpp")

	plt.legend()

	plt.show()
	plt.close()

## Correlation analysis

In [ ]:
correlations = ecgDf.corr(method = "pearson")

print(correlations)

In [ ]:

figure, axes = plt.subplots()

image = axes.imshow(correlations, aspect = "auto")

axes.set_title("Correlação entre as derivações")

axes.set_xticks(np.arange(12), labels = ecgHeaders[1:])
axes.set_yticks(np.arange(12), labels = ecgHeaders[1:])

for i in range(correlations.shape[0]):
	for j in range(correlations.shape[1]):
		correlation = np.round(
			correlations.iloc[i, j], 
			1
		)

		axes.text(
			i, j, 
			correlation, 
			ha = "center", 
			va = "center", 
			color = "red" if correlation == 0 else "w"
		)

plt.colorbar(image, ax = axes)

plt.show()
plt.close()


## aVR Linear Regression using LI and LII

$$ aVR_{rec} = - \frac{LI + LII}{\sqrt{3}} = -\frac{1}{\sqrt{3}}LI - \frac{1}{\sqrt{3}}LII $$

In [ ]:
linearRegressionModel = LinearRegression()

In [ ]:
linearModel = linearRegressionModel.fit(
	ecgDf[["LI", "LII"]],
	ecgDf["aVR"]
)

print(np.round(linearModel.coef_, 3))
print(np.round(linearModel.intercept_, 3))


In [ ]:
aVRRec = linearModel.predict(ecgDf[["LI", "LII"]])
aVRRec = pd.Series(aVRRec, ecgDf.index)

In [ ]:
aVRRecCorr = ecgDf["aVR"].corr(
	other = aVRRec, 
	method = "pearson"
)

aVRRecCorr = np.round(aVRRecCorr, 3)

print(aVRRecCorr)

In [ ]:
comparativeEcgPlot(
	ecg1 = aVRRec,
	label1 = "$aVR_{rec}$",
	
	ecg2 = ecgDf["aVR"],
	label2 = "aVR",
	
	title = "$aVR_{rec}$ vs aVR. Correlation = " + str(aVRRecCorr)
)

### aVR Linear Regression using LI and LiI considering only the QRS complex

In [ ]:

figure, axes = plt.subplots(
	nrows = 2, 
	ncols = 1,
	sharex = True,
)

figure.suptitle("Complexos QRS")
figure.supxlabel("Amostras")
figure.supylabel("Dpp")

axes[0].plot(ecgDf["LI"][100: 500], color = "blue")
axes[0].set_title("LI")

axes[1].plot(ecgDf["LII"][100: 500], color = "red")
axes[1].set_title("LII")

plt.tight_layout()

plt.show()

plt.close()

In [ ]:
linearQRSModel = linearRegressionModel.fit(
	ecgDf[["LI", "LII"]][100: 500],
	ecgDf["aVR"][100: 500]
)

print(np.round(linearQRSModel.coef_, 3))
print(np.round(linearQRSModel.intercept_, 3))


In [ ]:
aVRRecQRS = linearQRSModel.predict(ecgDf[["LI", "LII"]])
aVRRecQRS = pd.Series(aVRRecQRS, ecgDf.index)

In [ ]:
aVRRecQRSCorr = ecgDf["aVR"].corr(
	other = aVRRecQRS, 
	method = "pearson"
)

aVRRecQRSCorr = np.round(aVRRecQRSCorr, 3)

print(aVRRecQRSCorr)

In [ ]:
comparativeEcgPlot(
	ecg1 = aVRRecQRS,
	label1 = "$aVR_{QRS-Rec}$",
	
	ecg2 = ecgDf["aVR"],
	label2 = "aVR",
	
	title = "$aVR_{QRS-Rec}$ vs aVR. Correlation = " + str(aVRRecQRSCorr)
)

## ECG synthesis using the best set of leads (I, II, V2) from the neiwan article (2004)

### Plot Ecg Function

In [ ]:
def plotECG(ecgDf): 
	figure, axes = plt.subplots(
		nrows = 3,
		ncols = 4,
		sharex = True,
		figsize = (16, 9)
	)

	figure.suptitle("ECG 12-Lead")
	figure.supxlabel("Amostra")
	figure.supylabel("Dpp")

	axes[0, 0].plot(ecgDf["LI"], color = "#00916E")
	axes[0, 0].set_title("LI")

	axes[0, 1].plot(ecgDf["aVR"], color = "black")
	axes[0, 1].set_title("aVR")

	axes[0, 2].plot(ecgDf["V1"], color = "#FFCF00")
	axes[0, 2].set_title("V1")

	axes[0, 3].plot(ecgDf["V4"], color = "#EE6123")
	axes[0, 3].set_title("V4")


	axes[1, 0].plot(ecgDf["LII"], color = "#80A4ED")
	axes[1, 0].set_title("LII")

	axes[1, 1].plot(ecgDf["aVL"], color = "#00916E")
	axes[1, 1].set_title("aVL")

	axes[1, 2].plot(ecgDf["V2"], color = "#FFCF00")
	axes[1, 2].set_title("V2")

	axes[1, 3].plot(ecgDf["V5"], color = "#FA003F")
	axes[1, 3].set_title("V5")


	axes[2, 0].plot(ecgDf["LII"], color = "#80A4ED")
	axes[2, 0].set_title("LII")

	axes[2, 1].plot(ecgDf["aVF"], color = "#80A4ED")
	axes[2, 1].set_title("aVF")

	axes[2, 2].plot(ecgDf["V3"], color = "#EE6123")
	axes[2, 2].set_title("V3")

	axes[2, 3].plot(ecgDf["V6"], color = "#FA003F")
	axes[2, 3].set_title("V6")

	plt.tight_layout(pad = 1.5)

	plt.show()

	plt.close()

In [ ]:
plotECG(ecgDf)

In [ ]:
plotECG(ecgDf[100: 500])

In [ ]:
ecgRecDf = pd.DataFrame().reindex_like(ecgDf)

print(ecgRecDf.head())

In [ ]:
ecgRecDf["LI"] = ecgDf["LI"]
ecgRecDf["LII"] = ecgDf["LII"]
ecgRecDf["V2"] = ecgDf["V2"]

print(ecgRecDf.head())

In [ ]:
recHeaders = set(ecgHeaders) - set(["LI", "LII", "V2", "sample"])
print(recHeaders)

In [ ]:
regressionDf = pd.DataFrame(
	columns = ["lead", "LI", "LII", "V2", "Intercept"],
)

for recLead in recHeaders:
	recLinearModel = linearRegressionModel.fit(
		X = ecgDf[["LI", "LII", "V2"]][100: 500], 
		y = ecgDf[recLead][100: 500]
	)
	
	regressionDf.loc[len(regressionDf)] = [
		recLead,
		recLinearModel.coef_[0],
		recLinearModel.coef_[1],
		recLinearModel.coef_[2],
		recLinearModel.intercept_
	]

	recLeadSeries = recLinearModel.predict(X = ecgDf[["LI", "LII", "V2"]])
	recLeadSeries = pd.Series(recLeadSeries, index = ecgDf.index)
	
	ecgRecDf[recLead] = recLeadSeries

regressionDf = regressionDf.set_index("lead")

print(ecgRecDf.head())
print(regressionDf.head())


In [ ]:
plotECG(ecgRecDf)

In [ ]:
plotECG(ecgRecDf[100: 500])

In [ ]:
def comparativeFullEcgPlot(ecgA, ecgB):
	figure, axes = plt.subplots(
		nrows = 3,
		ncols = 4,
		sharex = True,
		figsize = (16, 9)
	)

	figure.suptitle("ECG 12-Lead")
	figure.supxlabel("Amostra")
	figure.supylabel("Dpp")

	axes[0, 0].plot(ecgA["LI"], color = "blue", alpha = 0.75)
	axes[0, 0].plot(ecgB["LI"], color = "red", alpha = 0.75)
	axes[0, 0].set_title("LI")

	axes[0, 1].plot(ecgA["aVR"], color = "blue", alpha = 0.75)
	axes[0, 1].plot(ecgB["aVR"], color = "red", alpha = 0.75)
	axes[0, 1].set_title("aVR")

	axes[0, 2].plot(ecgA["V1"], color = "blue", alpha = 0.75)
	axes[0, 2].plot(ecgB["V1"], color = "red", alpha = 0.75)
	axes[0, 2].set_title("V1")

	axes[0, 3].plot(ecgA["V4"], color = "blue", alpha = 0.75)
	axes[0, 3].plot(ecgB["V4"], color = "red", alpha = 0.75)
	axes[0, 3].set_title("V4")


	axes[1, 0].plot(ecgA["LII"], color = "blue", alpha = 0.75)
	axes[1, 0].plot(ecgB["LII"], color = "red", alpha = 0.75)
	axes[1, 0].set_title("LII")

	axes[1, 1].plot(ecgA["aVL"], color = "blue", alpha = 0.75)
	axes[1, 1].plot(ecgB["aVL"], color = "red", alpha = 0.75)
	axes[1, 1].set_title("aVL")

	axes[1, 2].plot(ecgA["V2"], color = "blue", alpha = 0.75)
	axes[1, 2].plot(ecgB["V2"], color = "red", alpha = 0.75)
	axes[1, 2].set_title("V2")

	axes[1, 3].plot(ecgA["V5"], color = "blue", alpha = 0.75)
	axes[1, 3].plot(ecgB["V5"], color = "red", alpha = 0.75)
	axes[1, 3].set_title("V5")


	axes[2, 0].plot(ecgA["LII"], color = "blue", alpha = 0.75)
	axes[2, 0].plot(ecgB["LII"], color = "red", alpha = 0.75)
	axes[2, 0].set_title("LII")

	axes[2, 1].plot(ecgA["aVF"], color = "blue", alpha = 0.75)
	axes[2, 1].plot(ecgB["aVF"], color = "red", alpha = 0.75)
	axes[2, 1].set_title("aVF")

	axes[2, 2].plot(ecgA["V3"], color = "blue", alpha = 0.75)
	axes[2, 2].plot(ecgB["V3"], color = "red", alpha = 0.75)
	axes[2, 2].set_title("V3")

	axes[2, 3].plot(ecgA["V6"], color = "blue", alpha = 0.75)
	axes[2, 3].plot(ecgB["V6"], color = "red", alpha = 0.75)
	axes[2, 3].set_title("V6")

	plt.tight_layout(pad = 1.5)

	plt.show()

	plt.close()

In [ ]:
comparativeFullEcgPlot(ecgDf, ecgRecDf)

In [ ]:
comparativeFullEcgPlot(
    ecgDf[100: 500], 
    ecgRecDf[100: 500]
)

In [ ]:

figure, axes = plt.subplots()

axes.axis("off")

figure.suptitle("Tabelas das regressões usadas para cada derivação")

figure.subplots_adjust(top = .5)

plt.table(
    cellText = np.round(regressionDf.values, 5),
    colLabels = regressionDf.columns,
    
    rowLabels = regressionDf.index,
    
	loc = "center",
)

plt.tight_layout(pad = 1.5)

plt.show()

plt.close()